In [ ]:

#--------------------------------- Librerias -----------------------------------

import numpy as np
import random
import math
import cmath
import pandas as pd

from numpy import linalg as LA

import matplotlib
from matplotlib import pyplot as plt
import matplotlib.animation

import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

from numpy.linalg import norm
from matplotlib.hatch import HorizontalHatch
import os
import csv

#------------------------------- General Config --------------------------------

IDEAL = False
DELTAt = 0
THERMOSTAT = 'None'

N = 2*2
r_2 = 0.8         #Disc 2 RatioTemp
r_1 = 0.4         #Disc 1 Ratio
r_G = 0.5         #Gap Ratio
delta_r = r_G-r_1 #Gap thickness

R = r_1*0.06

# N = 2*2
# r_2 = 8.0         #Disc 2 RatioTemp
# r_1 = 4.0         #Disc 1 Ratio
# r_G = 5.0         #Gap Ratio
# delta_r = r_G-r_1 #Gap thickness

# R = r_1*0.015

m = 1

Q_p = 1.0
Q_n = -Q_p

kB = 1

e0 = 1

Temp_Target = 1

Vm = list(np.arange(0,N))
Ke_Total_List = []
Ke_Mean_List = []
Pe_T_List = []
Vx_Mean_vec = []
Vy_Mean_vec = []
Temperature_Vec = []
VmComulate = []
Vb = list(np.arange(0,N))

Sqrt_Vel_Mean = 0

F = np.column_stack((np.zeros(N).tolist(),np.zeros(N).tolist()))
Fp = np.zeros(N).tolist()

#Nose-Hover-Verlet parameters
zeta = 0.0 #Initial zeta
xi = 0.1 #Relaxation parateter of the friction

#Langevin Thermostat
LambdaLG = 0.1

#Andersen Thermostat
GAMMA = 1

Ke_T = 0 #Energía cinética total
Ke_M = 0 #Energía cinética media

Pe_T = 0

ParticleData = pd.DataFrame()
SystemData = pd.DataFrame()

Fm = np.zeros(N).tolist()

Ke_p = np.zeros(N).tolist()

Step_List = []

#-------------------------------- Frontiers Plot -------------------------------

def plotFrontiers( show ) :
    figA, axFrontier = plt.subplots(dpi=141)
    axFrontier.set_aspect(1)
    axFrontier.set_xlim(-r_2-(0.05*r_2),r_2+(0.05*r_2))
    axFrontier.set_ylim(-r_2-(0.05*r_2),r_2+(0.05*r_2))
                
    plt.gca().set_aspect('equal') #Aspect ratio

    circle1 = plt.Circle((0,0), r_1, color='r')
    circle2 = plt.Circle((0,0), r_2, color='b')
    circleG = plt.Circle((0,0), r_G, color='r')
    
    pc = matplotlib.collections.PatchCollection([circle1,circle2,circleG],facecolor='none', edgecolor='black')
    
    axFrontier.add_collection(pc)    

    plt.xlabel("x")
    plt.ylabel("y")        
    if show : 
        plt.show()

#--------------------- Random nSteps coordinate pair values --------------------

#Generates nSteps random real numbers in [x1, x2]
def randomSet (x1, x2, nSteps ) : 
    randomlist = []
    for i in range(1, nSteps+1):
        number = random.uniform(x1, x2)
        randomlist.append(number)
    return np.array(randomlist) #Return a vector instead of a list   

def computeSize():
  figA, ax = plt.subplots(2,figsize=(4, 6),gridspec_kw={'height_ratios': [0.8, 0.2]},dpi=150)
  ax[0].set_aspect(1)
  ax[0].set_xlim(-r_2-(0.05*r_2),r_2+(0.05*r_2))
  ax[0].set_ylim(-r_2-(0.05*r_2),r_2+(0.05*r_2))
  particleSize = ((2*R*ax[0].get_window_extent().width  / (2*(4/np.pi)*(r_2+(0.1*r_2))) * 72./figA.dpi) ** 2)
  return particleSize

particleSize = computeSize()

In [ ]:

#--------------- Generates random N/2 particles into r_1 circle ----------------

def CirclePoints (r,nSteps): 
    X_randomlist = []
    Y_randomlist = []
    for i in range(1, nSteps+1):
        Relation = random.uniform(0.0,1.0)
        Angle = random.uniform(0.0,1.0)*2*3.1416 #Generate a radom angle between 0,2PI
        x_number = (r*0.95)*math.cos(Angle) * Relation
        y_number = (r*0.95)*math.sin(Angle) * Relation

        X_randomlist.append(x_number)
        Y_randomlist.append(y_number)

    return np.array(X_randomlist),np.array(Y_randomlist) #Return a vector instead of a list

#------------- Generates random N/2 particles between r_G and R_2 --------------

def DonutPoints (nSteps): 
    X_randomlist = []
    Y_randomlist = []
    for i in range(1, nSteps+1):
        Ratio = random.uniform(r_G+(r_G*0.05),r_2-(r_2*0.05))
        Angle = random.uniform(0.0,1.0)*2*3.1416 #Generate a radom angle between 0,2PI
        x_number = Ratio*math.cos(Angle)
        y_number = Ratio*math.sin(Angle)

        X_randomlist.append(x_number)
        Y_randomlist.append(y_number)

    return np.array(X_randomlist),np.array(Y_randomlist) #Return a vector instead of a list

#-- Gen random particles positions: N/2 into r_1 and N/2 between r_G and r_2 --

def ParticlePos(show) : 
    global X, Y, Vx, Vy, IDEAL
    
    #POSITIONS
    Xr2,Yr2 = DonutPoints(int(N/2))
    Xr1,Yr1 = CirclePoints(r_1,int(N/2))

    XList = Xr1.tolist() + Xr2.tolist()
    YList = Yr1.tolist() + Yr2.tolist()
    X = np.array(XList)
    Y = np.array(YList)
    #---------------------------------------------------------------------------
    
    #Random Velocities
    if IDEAL == True:
        Vx = randomSet(-1,1,int(N))    
        Vy = randomSet(-1,1,int(N)) 
        
    else:
        Vx = np.zeros(N)
        Vy = np.zeros(N)

    if show :
        print("Initial random Positions")
        print("X=", X,len(X))
        print("Y=", Y,len(Y))
        print("Vx=", Vx, len(Vx))
        print("Vy=", Vy, len(Vy))
       
#------------------ Plot particles with their random positions -----------------

def plotParticles(show) : 
    global X, Y, Vx, Vy

    XList = X.tolist()
    YList = Y.tolist()

    Xr1_List = XList[0 : int(N/2)]
    Yr1_List = YList[0 : int(N/2)]

    Xr2_List = XList[int(N/2) : int(N)]
    Yr2_List = YList[int(N/2) : int(N)] 

    ax[0].scatter(Xr2_List, Yr2_List, c ="yellow",
            linewidths = 1,
            marker ="o",
            edgecolor ="black",
            s = particleSize)
    
    ax[0].scatter(Xr1_List, Yr1_List, c ="red",
            linewidths = 1,
            marker ="o",
            edgecolor ="black",
            s = particleSize)


In [ ]:
# Frontier Collition detection (return state and ratio of collition frontier)

def frontierCollition(n,Xp,Yp) :
  global X, Y
  rCoord=np.sqrt(Xp**2 + Yp**2)
  
  #Collition with C1
  if n < int(N/2) : 
    if rCoord >= r_1 :
      return True,r_1
    else :
      return False,r_1

  #Collition with CG or C2  
  if n >= int(N/2):
    if rCoord <= r_G:
      return True,r_G
    elif rCoord >= r_2:
      return True,r_2
    else :
      return False,0

#------------------------------- Angle calculator ------------------------------

def Angle_Cal_Origin (X_Pos,Y_Pos):

  if X_Pos == 0.0 and Y_Pos == 0.0: #Si no hay desplazamiento, retorna un ángulo 0
      Angle = 0
  if X_Pos == 0.0 and Y_Pos > 0.0:  #Si sólo hay movimiento en Y positivo, retorna pi/2
    Angle = (math.pi)/2
  if X_Pos == 0.0 and Y_Pos < 0.0:  #Si sólo hay movimiento en Y negativo, retorna -pi/2
    Angle = (-1*(math.pi))/2
  if Y_Pos == 0.0 and X_Pos > 0.0:  #Si sólo hay movimiento en X positivo, retorna 0
    Angle = 0
  if Y_Pos == 0.0 and X_Pos < 0.0:  #Si sólo hay movimiento en X negativo, retorna pi
    Angle = math.pi
  
  abs_X_Pos = abs(X_Pos)
  abs_Y_Pos = abs(Y_Pos)

  #Cálculo dinámico de ángulos de movimiento
  if X_Pos > 0.0 and Y_Pos > 0.0:                 # Cuadrante 1
    Angle = math.atan(abs_Y_Pos/abs_X_Pos)
  if X_Pos < 0.0 and Y_Pos > 0.0:                 # Cuadrante 2
    Angle = ((math.pi)/2)+math.atan(abs_X_Pos/abs_Y_Pos)
  if X_Pos < 0.0 and Y_Pos < 0.0:                 # Cuadrante 3
    Angle = (-1*(math.pi)/2)-math.atan(abs_X_Pos/abs_Y_Pos)
  if X_Pos > 0.0 and Y_Pos < 0.0:                 # Cuadrante 4
    Angle = -1*math.atan(abs_Y_Pos/abs_X_Pos)

  return Angle

#------------------------- Collition Point calculator --------------------------

def CollitionPoint (Last_X,Last_Y,Next_X,Next_Y,Frontera):

  Delta_Y = Next_Y-Last_Y
  Delta_X = Next_X-Last_X

  if Delta_X == 0.0 or Delta_Y==0.0:
    return Last_X,Last_Y,0

  m = (Delta_Y)/(Delta_X)
  b = Last_Y-(m*Last_X)

  A = 1+(m**2)
  B = 2*b*m
  C = ((b**2)-(Frontera**2))
  
  try:
    Choque_Xp = (-B+(math.sqrt((B**2)-(4*A*C))))/(2*A)  
    Choque_Xn = (-B-(math.sqrt((B**2)-(4*A*C))))/(2*A)  
    
  except Exception as e:
    
    Choque_Xp = (-B+(cmath.sqrt((B**2)-(4*A*C))))/(2*A)
    Choque_Xn = (-B-(cmath.sqrt((B**2)-(4*A*C))))/(2*A)
    
    Choque_Xp = Choque_Xp.real
    Choque_Xn = Choque_Xn.real

  Choque_Yp = (m*Choque_Xp)+b
  Choque_Yn = (m*Choque_Xn)+b

  Ratio_p = math.sqrt((Choque_Xp**2)+(Choque_Yp**2))
  Ratio_n = math.sqrt((Choque_Xn**2)+(Choque_Yn**2))

  Ratio_p = round(Ratio_p,2)
  Ratio_n = round(Ratio_n,2)

  Delta_Choque_Xp = Choque_Xp - Last_X
  Delta_Choque_Yp = Choque_Yp - Last_Y

  Delta_Choque_Xn = Choque_Xn - Last_X
  Delta_Choque_Yn = Choque_Yn - Last_Y

  Vec_Delta = [Delta_X,Delta_Y]
  Vec_ChoqP = [Delta_Choque_Xp,Delta_Choque_Yp]
  Vec_ChoqN = [Delta_Choque_Xn,Delta_Choque_Yn]

  Dot_p = np.dot(Vec_Delta,Vec_ChoqP)
  Dot_n = np.dot(Vec_Delta,Vec_ChoqN)

  if Frontera == r_1 or Frontera == r_2:
    if Dot_p >= 0:
      Angle = Angle_Cal_Origin(Choque_Xp,Choque_Yp)
      return Choque_Xp,Choque_Yp,Angle

    elif Dot_n >= 0:
      Angle = Angle_Cal_Origin(Choque_Xn,Choque_Yn)
      return Choque_Xn,Choque_Yn,Angle

    elif Dot_p > Dot_n:
      Angle = Angle_Cal_Origin(Choque_Xp,Choque_Yp)
      return Choque_Xp,Choque_Yp,Angle
      
    elif Dot_n > Dot_p:
      Angle = Angle_Cal_Origin(Choque_Xn,Choque_Yn)
      return Choque_Xn,Choque_Yn,Angle
    
  else:
    if Dot_p <= 0:
      Angle = Angle_Cal_Origin(Choque_Xp,Choque_Yp)
      return Choque_Xp,Choque_Yp,Angle

    elif Dot_n <= 0:
      Angle = Angle_Cal_Origin(Choque_Xn,Choque_Yn)
      return Choque_Xn,Choque_Yn,Angle

    elif Dot_p < Dot_n:
      Angle = Angle_Cal_Origin(Choque_Xp,Choque_Yp)
      return Choque_Xp,Choque_Yp,Angle

    elif Dot_n < Dot_p:
      Angle = Angle_Cal_Origin(Choque_Xn,Choque_Yn)
      return Choque_Xn,Choque_Yn,Angle
    
  if Vec_ChoqN == Vec_ChoqP:
    Angle = Angle_Cal_Origin(Choque_Xp,Choque_Yp)
    return Choque_Xp,Choque_Yp,Angle

#------------------ Calculator of Velocities after Collition -------------------

def newVelocities( VxOld, VyOld, phi_c ) :
  VxNew = -VxOld*np.cos(2*phi_c)-VyOld*np.sin(2*phi_c)
  VyNew = -VxOld*np.sin(2*phi_c)+VyOld*np.cos(2*phi_c)
  return VxNew, VyNew


In [ ]:
from numpy.linalg import norm

def hardDiskCollision(rVecList1, rVecList2, uVecList, vVecList) :    
  x1, y1 = rVecList1
  x2, y2 = rVecList2
  uxVal, uyVal = uVecList
  vxVal, vyVal = vVecList
          
  rVector1 = np.array(rVecList1) 
  rVector2 = np.array(rVecList2) 
  uVector = np.array(uVecList) 
  vVector = np.array(vVecList) 

  rVector12 = rVector2-rVector1
  rVec12Distance = norm(rVector12)
  rVector12Unit = rVector12/rVec12Distance

  cosphi, sinphi = rVector12[0]/rVec12Distance, rVector12[1]/rVec12Distance

  #Unit vectors of the cartesian coordinate system
  xUnit = np.array([1, 0])
  yUnit = np.array([0, 1])

  #Rotation matrix and its inverse
  RMatrix = np.array([[cosphi, -sinphi], [sinphi, cosphi]])
  RMatrixInv = np.array([[cosphi, sinphi], [-sinphi, cosphi]])
      

  #Step 1. Unit vectors in the rotated system
  xUnitTilde = np.dot(RMatrix,xUnit)   #xUnitTilde = np.array([cosphi, -sinphi])
  yUnitTilde = np.dot(RMatrix,yUnit)   #yUnitTilde = np.array([sinphi, cosphi])
      
  #Step 2. Velocities vectors in the rotated system
  uVectorTilde = np.dot(RMatrixInv,uVector)
  vVectorTilde = np.dot(RMatrixInv,vVector)
  #Components
  uxTilde, uyTilde = uVectorTilde[0], uVectorTilde[1]
  vxTilde, vyTilde = vVectorTilde[0], vVectorTilde[1]
      
  #New velocities (in the non-rotated cartesian system)
  uNewVectorTilde = vxTilde*xUnitTilde + uyTilde*yUnitTilde
  vNewVectorTilde = uxTilde*xUnitTilde + vyTilde*yUnitTilde

  uNewVector = uNewVectorTilde
  vNewVector = vNewVectorTilde

  #New positions
  rVectorMid = (rVector1+rVector2)/2.0  #middle position where paticles collide
  rNewVector1 = rVectorMid - 1.01*R*rVector12Unit 
  rNewVector2 = rVectorMid + 1.01*R*rVector12Unit 

  x1, y1 = rNewVector1.tolist()
  x2, y2 = rNewVector2.tolist()
  uxVal, uyVal = uNewVector.tolist()
  vxVal, vyVal = vNewVector.tolist()    
    
  return rNewVector1.tolist(), rNewVector2.tolist(), uNewVector.tolist(), vNewVector.tolist()

def checkCollisions():
    
    X_mat = np.array(X).reshape(N, 1)
    Y_mat = np.array(Y).reshape(N, 1)
    
    X_mat = X_mat - X_mat.T
    Y_mat = Y_mat - Y_mat.T
    Vx_mat = np.array(Vx).reshape(N, 1) - np.array(Vx).reshape(N, 1).T
    Vy_mat = np.array(Vy).reshape(N, 1) - np.array(Vy).reshape(N, 1).T
    
    dist_mat = np.sqrt(X_mat**2 + Y_mat**2)
    overlap_mat = dist_mat < 2 * R
    
    collision_indices = np.triu(overlap_mat, k=1).nonzero()
    
    for i, n in zip(*collision_indices):
        XiNew, YiNew, VxiNew, VyiNew = X[i], Y[i], Vx[i], Vy[i]
        XnNew, YnNew, VxnNew, VynNew = X[n], Y[n], Vx[n], Vy[n]
        
        [X[i],Y[i]],[X[n],Y[n]],[Vx[i],Vy[i]],[Vx[n],Vy[n]] = hardDiskCollision([XiNew, YiNew], [XnNew, YnNew], [VxiNew, VyiNew], [VxnNew, VynNew])


In [ ]:
from matplotlib.hatch import HorizontalHatch

def Velocities_Mag ():
  global Vx, Vy, Vm

  for n in range(0, int(N)) :
    Vm[n] = math.sqrt(((Vx[n])**2)+((Vy[n])**2))

  Vm = np.array(Vm)
  Mean = np.mean(Vm**2)
  
  return Mean

def XVel_Mean ():
  global Vx
  return np.mean(Vx)

def YVel_Mean ():
  global Vy
  return np.mean(Vy)

def Mean_Kinetic_Energy ():
  global Sqrt_Vel_Mean

  K = 0.5*m*Sqrt_Vel_Mean
  return K

def Temperature_Cal (K):
  global kB
  
  T = K/kB
  return T

#------------- Movement funtion: Update positions and velocitites --------------

def magnitude(vector) : 
    return LA.norm(vector)

def q(n) :
    if IDEAL == True:
      return 0
    else: 
      if n < N/2: 
        return Q_p # n will start at zero
      else : 
        return Q_n

def Coulomb_Force_Cal(X, Y):
  F = []
  rVector = np.column_stack((X, Y))  # vector de posición
  # q_array = np.array([q(i) for i in range(int(N))])
  q_array = np.array([Q_p if i < N/2 else Q_n for i in range(int(N))])
  q_matrix = q_array[:, np.newaxis]
  r_ij = rVector[:, np.newaxis, :] - rVector
  r_ij_mag = np.linalg.norm(r_ij, axis=2)
  np.fill_diagonal(r_ij_mag, 1.0)  # Evitar la división por cero en la diagonal
  
  r_ij_hat = r_ij / r_ij_mag[:, :, np.newaxis]
  F_matrix = (q_matrix * q_matrix.T)[:, :, np.newaxis] * r_ij_hat / r_ij_mag[:, :, np.newaxis]**2
  Conts = 1 / (4 * np.pi * e0)
  F = np.sum(F_matrix*Conts, axis=1)

  return F.tolist()

def Potencial_Energy(x,y):
    rVector = np.column_stack((x, y))  # posición vectorial
    Conts = 1 / (4 * np.pi * e0)
    Q = np.array([q(n) for n in range(N)])

    # Calcula las distancias de manera eficiente usando broadcasting
    r_diff = rVector[:, np.newaxis, :] - rVector
    distances = np.linalg.norm(r_diff, axis=2)
    np.fill_diagonal(distances, 1.0)  # Evitar la división por cero para las partículas coincidentes

    # Calcula la energía potencial usando cálculos matriciales
    energy_matrix = Q[:, np.newaxis] * Q * Conts / distances
    np.fill_diagonal(energy_matrix, 0.0)  # Excluye la contribución propia

    Sum_m = np.sum(energy_matrix)

    return Sum_m

def moveD() :           # Sin Termostato
  global X, Y, Vx, Vy,F,Fp, DELTAt, IDEAL, F

  if IDEAL == False:
    #Approx : Velocity Verlet integrators 
    F=np.vstack(Coulomb_Force_Cal(X,Y))  #Compute the forces. F[i] is the force on the ith-particle 
    ax=F[:,0]/m               #Total x-acceleration on each particle
    ay=F[:,1]/m               #Total y-acceleration on each particle
    #New position
    Xp = X+Vx*DELTAt + 0.5*ax*DELTAt**2
    Yp = Y+Vy*DELTAt + 0.5*ay*DELTAt**2
    #New acceleration
    Fp=np.vstack(Coulomb_Force_Cal(Xp,Yp)) #Compute the forces. F[i] is the force on the ith-particle 
    axp=Fp[:,0]/m #Total x-acceleration on each particle
    ayp=Fp[:,1]/m #Total y-acceleration on each particle
    #Verlet velocity
    Vxp = Vx + 0.5*(ax+axp)*DELTAt
    Vyp = Vy + 0.5*(ay+ayp)*DELTAt
    
    for n in range(0, int(N)) :
      Collition,Front = frontierCollition(n,Xp[n],Yp[n])
      if Collition == False:
        X[n] = Xp[n]
        Y[n] = Yp[n]
        Vx[n] = Vxp[n]
        Vy[n] = Vyp[n]
      else:
        X[n],Y[n],Angle_c=CollitionPoint (X[n],Y[n],Xp[n],Yp[n],Front)
        Vx[n], Vy[n] = newVelocities( Vx[n], Vy[n], Angle_c ) #update
        X[n] = X[n]+Vx[n]*DELTAt 
        Y[n] = Y[n]+Vy[n]*DELTAt

  else:
    Xp = X+Vx*DELTAt 
    Yp = Y+Vy*DELTAt
    
    Vxp = Vx 
    Vyp = Vy 
        
    for n in range(0, int(N)) :
      Collition,Front = frontierCollition(n,Xp[n],Yp[n])
      if Collition == False:
        X[n] = Xp[n]
        Y[n] = Yp[n]
        Vx[n] = Vxp[n]
        Vy[n] = Vyp[n]
      else:
        X[n],Y[n],Angle_c=CollitionPoint (X[n],Y[n],Xp[n],Yp[n],Front)
        Vx[n], Vy[n] = newVelocities( Vx[n], Vy[n], Angle_c ) #update
        X[n] = X[n]+Vx[n]*DELTAt 
        Y[n] = Y[n]+Vy[n]*DELTAt
    
# Reescalado de velocidades
def Vel_Rescaling(Velx,Vely,Temp_Act):
  global Temp_Target
  
  V_x = Velx * math.sqrt(Temp_Target/Temp_Act)
  V_y = Vely * math.sqrt(Temp_Target/Temp_Act)
  
  return V_x,V_y
    
def moveVR():
  global X, Y, Vx, Vy,F,Fp, DELTAt, IDEAL, Sqrt_Vel_Mean, F

  if IDEAL == False:
    #Approx : Velocity Verlet integrators 
    F=np.vstack(Coulomb_Force_Cal(X,Y))  #Compute the forces. F[i] is the force on the ith-particle 
    ax=F[:,0]/m               #Total x-acceleration on each particle
    ay=F[:,1]/m               #Total y-acceleration on each particle
    #New position
    Xp = X+Vx*DELTAt + 0.5*ax*DELTAt**2
    Yp = Y+Vy*DELTAt + 0.5*ay*DELTAt**2
    #New acceleration
    Fp=np.vstack(Coulomb_Force_Cal(Xp,Yp)) #Compute the forces. F[i] is the force on the ith-particle 
    axp=Fp[:,0]/m #Total x-acceleration on each particle
    ayp=Fp[:,1]/m #Total y-acceleration on each particle
    #Verlet velocity
    Vxp = Vx + 0.5*(ax+axp)*DELTAt
    Vyp = Vy + 0.5*(ay+ayp)*DELTAt
    
    for n in range(0, int(N)) :
      Collition,Front = frontierCollition(n,Xp[n],Yp[n])
      if Collition == False:
        X[n] = Xp[n]
        Y[n] = Yp[n]
        Vx[n] = Vxp[n]
        Vy[n] = Vyp[n]
      else:
        X[n],Y[n],Angle_c=CollitionPoint (X[n],Y[n],Xp[n],Yp[n],Front)
        Vx[n], Vy[n] = newVelocities( Vx[n], Vy[n], Angle_c ) #update
        X[n] = X[n]+Vx[n]*DELTAt 
        Y[n] = Y[n]+Vy[n]*DELTAt

  else:
    Xp = X+Vx*DELTAt 
    Yp = Y+Vy*DELTAt
    
    Vxp = Vx 
    Vyp = Vy 
        
    for n in range(0, int(N)) :
      Collition,Front = frontierCollition(n,Xp[n],Yp[n])
      if Collition == False:
        X[n] = Xp[n]
        Y[n] = Yp[n]
        Vx[n] = Vxp[n]
        Vy[n] = Vyp[n]
      else:
        X[n],Y[n],Angle_c=CollitionPoint (X[n],Y[n],Xp[n],Yp[n],Front)
        Vx[n], Vy[n] = newVelocities( Vx[n], Vy[n], Angle_c ) #update
        X[n] = X[n]+Vx[n]*DELTAt 
        Y[n] = Y[n]+Vy[n]*DELTAt
  
  #Velocity rescaling
  Sqrt_Vel_Mean = Velocities_Mag()
  Ke_M = Mean_Kinetic_Energy()
  T = Temperature_Cal(Ke_M)
  for n in range(0, int(N)) :
    Vx[n], Vy[n]=Vel_Rescaling(Vx[n],Vy[n],T)

def Total_KEnergy(Vx,Vy):    
  global m, N
  
  energySum = 0
  
  for i in range(0,N) :
      energySum = energySum + 0.5*m*((Vx[i]**2)+(Vy[i]**2))
  
  return energySum

def moveNH():
  global X, Y, Vx, Vy,F,Fp, DELTAt, IDEAL, zeta, xi, N, Ke_T, Ke_M, F
  
  if IDEAL == False:
    #Approx : Velocity Verlet integrators 
    F=np.vstack(Coulomb_Force_Cal(X,Y))  #Compute the forces. F[i] is the force on the ith-particle 
    ax=F[:,0]/m               #Total x-acceleration on each particle
    ay=F[:,1]/m               #Total y-acceleration on each particle
    
    #New position
    Xp = X + Vx*DELTAt + ((ax-zeta*Vx)*(0.5*(DELTAt**2)))
    Yp = Y + Vy*DELTAt + ((ay-zeta*Vy)*(0.5*(DELTAt**2)))

    Ke_T = Total_KEnergy(Vx,Vy)
    
    #At t + tDelta/2
    Vx_pp  = Vx + ((0.5*DELTAt)*(ax-(zeta*Vx)))           # Eq. 40
    Vy_pp  = Vy + ((0.5*DELTAt)*(ay-(zeta*Vy)))           # Eq. 40
    Ke_Tpp = Total_KEnergy(Vx_pp,Vy_pp)
    zetapp = zeta + (xi*0.5*DELTAt*(Ke_T-((((2*N)+1)*0.5)*kB*Temp_Target)))   # Eq. 36
    
    #At t + tDelta
    zetap = zetapp +(xi*0.5*DELTAt*(Ke_Tpp-((((2*N)+1)*0.5)*kB*Temp_Target)))  # Eq. 36
    Fp=np.vstack(Coulomb_Force_Cal(Xp,Yp))
    axp=Fp[:,0]/m
    ayp=Fp[:,1]/m
    #Verlet velocity with the news force
    Vxp = (Vx_pp + (0.5*DELTAt*axp))/(1+(0.5*DELTAt*zetap))
    Vyp = (Vy_pp + (0.5*DELTAt*ayp))/(1+(0.5*DELTAt*zetap))
    zeta = zetap #Update the value of zeta(t)
    
    for n in range(0, int(N)) :
      Collition,Front = frontierCollition(n,Xp[n],Yp[n])
      if Collition == False:
        X[n] = Xp[n]
        Y[n] = Yp[n]
        Vx[n] = Vxp[n]
        Vy[n] = Vyp[n]
      else:
        X[n],Y[n],Angle_c=CollitionPoint (X[n],Y[n],Xp[n],Yp[n],Front)
        Vx[n], Vy[n] = newVelocities( Vx[n], Vy[n], Angle_c ) #update
        X[n] = X[n]+Vx[n]*DELTAt 
        Y[n] = Y[n]+Vy[n]*DELTAt
            
  else:
    ax = np.zeros(N)
    ay = np.zeros(N)
    
    #New position
    Xp = X + Vx*DELTAt + ((ax-zeta*Vx)*(0.5*(DELTAt**2)))
    Yp = Y + Vy*DELTAt + ((ay-zeta*Vy)*(0.5*(DELTAt**2)))

    Ke_T = Total_KEnergy(Vx,Vy)
    
    #At t + tDelta/2
    Vx_pp  = Vx + ((0.5*DELTAt)*(ax-(zeta*Vx)))           # Eq. 40
    Vy_pp  = Vy + ((0.5*DELTAt)*(ay-(zeta*Vy)))           # Eq. 40
    Ke_Tpp = Total_KEnergy(Vx_pp,Vy_pp)
    zetapp = zeta + (xi*0.5*DELTAt*(Ke_T-((((2*N)+1)*0.5)*kB*Temp_Target)))   # Eq. 36
    
    #At t + tDelta
    zetap = zetapp +(xi*0.5*DELTAt*(Ke_Tpp-((((2*N)+1)*0.5)*kB*Temp_Target)))  # Eq. 36
    axp=np.zeros(N)
    ayp=np.zeros(N)
    #Verlet velocity with the news force
    Vxp = (Vx_pp + (0.5*DELTAt*axp))/(1+(0.5*DELTAt*zetap))
    Vyp = (Vy_pp + (0.5*DELTAt*ayp))/(1+(0.5*DELTAt*zetap))
    zeta = zetap #Update the value of zeta(t)
        
    for n in range(0, int(N)) :
      Collition,F = frontierCollition(n,Xp[n],Yp[n])
      if Collition == False:
        X[n] = Xp[n]
        Y[n] = Yp[n]
        Vx[n] = Vxp[n]
        Vy[n] = Vyp[n]
      else:
        X[n],Y[n],Angle_c=CollitionPoint (X[n],Y[n],Xp[n],Yp[n],F)
        Vx[n], Vy[n] = newVelocities( Vx[n], Vy[n], Angle_c ) #update
        X[n] = X[n]+Vx[n]*DELTAt 
        Y[n] = Y[n]+Vy[n]*DELTAt

def moveLG():
  global X, Y, Vx, Vy,F,Fp, DELTAt, IDEAL, LambdaLG, F

  if IDEAL == False:
    #Approx : Velocity Verlet integrators 
    F=np.vstack(Coulomb_Force_Cal(X,Y))  #Compute the forces. F[i] is the force on the ith-particle 
    ax=F[:,0]/m               #Total x-acceleration on each particle
    ay=F[:,1]/m               #Total y-acceleration on each particle
    #New position
    Xp = X+Vx*DELTAt + 0.5*ax*DELTAt**2
    Yp = Y+Vy*DELTAt + 0.5*ay*DELTAt**2
    #New acceleration
    Fp=np.vstack(Coulomb_Force_Cal(Xp,Yp)) #Compute the forces. F[i] is the force on the ith-particle 
    axp=Fp[:,0]/m #Total x-acceleration on each particle
    ayp=Fp[:,1]/m #Total y-acceleration on each particle
    #Verlet velocity
    Vxp = Vx + 0.5*(ax+axp)*DELTAt
    Vyp = Vy + 0.5*(ay+ayp)*DELTAt
    
    for n in range(0, int(N)) :
      Collition,Front = frontierCollition(n,Xp[n],Yp[n])
      if Collition == False:
        X[n] = Xp[n]
        Y[n] = Yp[n]
        Vx[n] = Vxp[n]
        Vy[n] = Vyp[n]
      else:
        X[n],Y[n],Angle_c=CollitionPoint (X[n],Y[n],Xp[n],Yp[n],Front)
        Vx[n], Vy[n] = newVelocities( Vx[n], Vy[n], Angle_c ) #update
        X[n] = X[n]+Vx[n]*DELTAt 
        Y[n] = Y[n]+Vy[n]*DELTAt

  else:
    Xp = X+Vx*DELTAt 
    Yp = Y+Vy*DELTAt
    
    Vxp = Vx 
    Vyp = Vy 
        
    for n in range(0, int(N)) :
      Collition,Front = frontierCollition(n,Xp[n],Yp[n])
      if Collition == False:
        X[n] = Xp[n]
        Y[n] = Yp[n]
        Vx[n] = Vxp[n]
        Vy[n] = Vyp[n]
      else:
        X[n],Y[n],Angle_c=CollitionPoint (X[n],Y[n],Xp[n],Yp[n],Front)
        Vx[n], Vy[n] = newVelocities( Vx[n], Vy[n], Angle_c ) #update
        X[n] = X[n]+Vx[n]*DELTAt 
        Y[n] = Y[n]+Vy[n]*DELTAt
  
  Vx = Vx - LambdaLG * Vx * DELTAt + np.sqrt(2 * LambdaLG * Temp_Target * DELTAt) * np.random.normal(0, 1)
  Vy = Vy - LambdaLG * Vy * DELTAt + np.sqrt(2 * LambdaLG * Temp_Target * DELTAt) * np.random.normal(0, 1)

def moveAD():
  global X, Y, Vx, Vy,F,Fp, DELTAt, IDEAL, Sqrt_Vel_Mean, GAMMA, F

  if IDEAL == False:
    #Approx : Velocity Verlet integrators 
    F=np.vstack(Coulomb_Force_Cal(X,Y))  #Compute the forces. F[i] is the force on the ith-particle 
    ax=F[:,0]/m               #Total x-acceleration on each particle
    ay=F[:,1]/m               #Total y-acceleration on each particle
    #New position
    
    Xp = X+Vx*DELTAt + 0.5*ax*DELTAt**2
    Yp = Y+Vy*DELTAt + 0.5*ay*DELTAt**2
    #New acceleration
    Fp=np.vstack(Coulomb_Force_Cal(Xp,Yp)) #Compute the forces. F[i] is the force on the ith-particle 
    axp=Fp[:,0]/m #Total x-acceleration on each particle
    ayp=Fp[:,1]/m #Total y-acceleration on each particle
    #Verlet velocity
    Vxp = Vx + 0.5*(ax+axp)*DELTAt
    Vyp = Vy + 0.5*(ay+ayp)*DELTAt
    
    # Termostato de Andersen
    for i in range(int(N)):
      if np.random.random() < GAMMA:
        Vxp[i] = np.random.normal(0, np.sqrt(Temp_Target))
        Vyp[i] = np.random.normal(0, np.sqrt(Temp_Target))
    
    for n in range(0, int(N)) :
      Collition,Front = frontierCollition(n,Xp[n],Yp[n])
      if Collition == False:
        X[n] = Xp[n]
        Y[n] = Yp[n]
        Vx[n] = Vxp[n]
        Vy[n] = Vyp[n]
      else:
        X[n],Y[n],Angle_c=CollitionPoint (X[n],Y[n],Xp[n],Yp[n],Front)
        Vx[n], Vy[n] = newVelocities( Vx[n], Vy[n], Angle_c ) #update
        X[n] = X[n]+Vx[n]*DELTAt 
        Y[n] = Y[n]+Vy[n]*DELTAt

  else:
    Xp = X+Vx*DELTAt 
    Yp = Y+Vy*DELTAt
    
    Vxp = Vx 
    Vyp = Vy 
    
    # Termostato de Andersen
    for i in range(int(N)):
      if np.random.random() < GAMMA:
        Vxp[i] = np.random.normal(0, np.sqrt(Temp_Target))
        Vyp[i] = np.random.normal(0, np.sqrt(Temp_Target))
        
    for n in range(0, int(N)) :
      Collition,Front = frontierCollition(n,Xp[n],Yp[n])
      if Collition == False:
        X[n] = Xp[n]
        Y[n] = Yp[n]
        Vx[n] = Vxp[n]
        Vy[n] = Vyp[n]
      else:
        X[n],Y[n],Angle_c=CollitionPoint (X[n],Y[n],Xp[n],Yp[n],Front)
        Vx[n], Vy[n] = newVelocities( Vx[n], Vy[n], Angle_c ) #update
        X[n] = X[n]+Vx[n]*DELTAt 
        Y[n] = Y[n]+Vy[n]*DELTAt

#---------------------------- Stack Movement steps ----------------------------

def animate(n1) :
  global X,Y,Sqrt_Vel_Mean,Ke_Mean_List,Vx_Mean_vec,Vy_Mean_vec

  ax[0].cla()
  ax[1].cla()

  circle1 = plt.Circle((0,0), r_1, color='r')
  circle2 = plt.Circle((0,0), r_2, color='b')
  circleG = plt.Circle((0,0), r_G, color='r')

  pc = matplotlib.collections.PatchCollection([circle1,circle2,circleG],facecolor='none', edgecolor='black')
  ax[0].add_collection(pc)    

  for the_counter in range(0, int(50)) :
    if THERMOSTAT == 'VR':
      moveVR()
    elif THERMOSTAT == 'NH':
      moveNH()
    elif THERMOSTAT == 'LG':
      moveLG()
    elif THERMOSTAT == 'AD':
      moveAD()
    else:
      moveD()
      
    checkCollisions()            

  ax[0].quiver(X.tolist(), Y.tolist(), Vx.tolist(), Vy.tolist(), color='b', units='xy', linewidths=0.4, width=0.02, scale=1, ls='solid',fc='w', edgecolor='black', alpha=0.2)

  ax[1] = sns.distplot(Vm,norm_hist=True)

  print("n1 =",n1+1, " of ", frameSteps)
  plotParticles (True)

In [ ]:

output_dir = r"E:\Cristian\1. Molecular Dinamics\2. Python Projects\3. Jupiter Code\7. Nose_Hoover Fixed\2. N_1000\20. N_2\T_0.085"
filename = "animacion.gif"
output_path = os.path.join(output_dir, filename)

# --------------------------- Simulation Conditions ---------------------------

PC_ID = "Equipo 3"
IDEAL = False
THERMOSTAT = 'NH'
DELTAt = 0.001
ANIMATE1 = True
ANIMATE2 = True

Temp_Target = 0.085

totalSteps = 20000000
Saving_Steps = 500
Saving_Steps_Str = str(Saving_Steps)

#Nose-Hover-Verlet parameters
zeta = 0.0 #Initial zeta

xi = 0.8
xi = ((xi/(2*np.pi))**2) * (2*(2*N)*kB*Temp_Target)
xi = 1/xi

Q_p = 1.0
Q_n = -Q_p

#------------------------------ Exec Intructions ------------------------------

ParticlePos (show = False)              # Generate random X,Y initial positions

#----------------------------- Animate Graph config ----------------------------

plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150  
plt.ioff()
fig, ax = plt.subplots(2,figsize=(4, 6),gridspec_kw={'height_ratios': [0.8, 0.2]})
frameSteps = 100

#------------ Exec movement with Animate funtion (stack movements) -------------

if ANIMATE1 == True:
    ani = matplotlib.animation.FuncAnimation(fig, animate, frames=frameSteps)
    ani.save(output_path, writer="Pillow")
    
# matplotlib.animation.FuncAnimation(fig, animate, frames=frameSteps)


In [ ]:
def create_csv_file(data_name, out_dir, columns):
    file_name = f"{data_name}.csv"
    file_path = os.path.join(out_dir, file_name)
    
    # columns = ['Step', 'Particle', 'X_Pos', 'Y_Pos', 'Vel_X', 'Vel_Y', 'Vel_M', 'Force_X', 'Force_Y', 'Force_T', 'Ke_Energy']
    
    with open(file_path, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(columns)

def add_data_to_csv(path, data):
    with open(path, 'a', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(data)

def add_long_lists_to_csv(path, column_indices, lists):
    rows = zip(*lists)  # Transpose lists to create rows
    
    with open(path, 'a', newline='') as csv_file:
        writer = csv.writer(csv_file)
        
        for row in rows:
            data = [''] * len(column_indices)  # Create empty data list
            
            for i, value in zip(column_indices, row):
                if isinstance(value, float):
                    formatted_value = "{:.9f}".format(value)
                    data[i] = formatted_value.replace('.', ',')
                else:
                    data[i] = value
            
            writer.writerow(data)

In [ ]:
fig3, ax3 = plt.subplots(dpi=100)

Particles = list(range(1, N+1))

List_Data1 = ['Step', 'Particle', 'X_Pos', 'Y_Pos', 'Vel_X', 
              'Vel_Y', 'Vel_M', 'Force_X', 'Force_Y', 'Force_T', 'Ke_Energy']

List_Data2 = ['Step', 'Ke_T', 'Ke_M', 'Temp_Act',
              'Temp_Tar', 'Vx_Mean', 'Vy_Mean','Pot_Energy']

# Índices de columna correspondientes
column_indices1 = list(range(len(List_Data1)))
column_indices2 = list(range(len(List_Data2)))

Par_data_name = 'Particle_Data'
csv_file_path = f"{output_dir}/{Par_data_name}.csv"
create_csv_file(Par_data_name, output_dir, List_Data1)

for the_counter in range(0, int(totalSteps)) :
  
  Last_Step = [the_counter+1] * N
  
  if THERMOSTAT == 'VR':
    moveVR()
  elif THERMOSTAT == 'NH':
    moveNH()
  elif THERMOSTAT == 'LG':
    moveLG()
  elif THERMOSTAT == 'AD':
    moveAD()
  else:
    moveD()
    
  Sqrt_Vel_Mean = Velocities_Mag()
  checkCollisions()

  if (the_counter+1)%Saving_Steps == 0:
    print("progress = ", the_counter*100.0/totalSteps)
    
    VmComulate = np.concatenate((VmComulate, Vm), axis=None)
    Step_List.append(the_counter+1)

    Sqrt_Vel_Mean = Velocities_Mag()
    Xvel_Mean = XVel_Mean()
    Vx_Mean_vec.append(Xvel_Mean)

    Yvel_Mean = YVel_Mean()
    Vy_Mean_vec.append(Yvel_Mean)

    Ke_T = Total_KEnergy(Vx,Vy)
    Ke_Total_List.append(Ke_T)
    
    Ke_Mean = Mean_Kinetic_Energy()
    Ke_Mean_List.append(Ke_Mean)
    
    Pe_T = Potencial_Energy(X,Y)
    Pe_T_List.append(Pe_T)
    
    T = Temperature_Cal (Ke_Mean)
    Temperature_Vec.append(T)
    
    for n in range(0, int(N)) :
      
      if IDEAL == True:
        F = np.zeros((N, 2))
      
      Vm[n] = math.sqrt(((Vx[n])**2)+((Vy[n])**2))
      Fm[n] = math.sqrt(((F[n,0])**2)+((F[n,1])**2))
      Ke_p[n] = 0.5*m*((Vx[n]**2)+(Vy[n]**2))

    add_long_lists_to_csv(csv_file_path, column_indices1, [Last_Step, Particles, X, Y, Vx, Vy, Vm, F[:,0], F[:,1], Fm, Ke_p])


In [ ]:
Temp_Tar_List = [Temp_Target] * len(Step_List)

Sys_data_name = 'System_Data'
csv_file_path = f"{output_dir}/{Sys_data_name}.csv"

create_csv_file(Sys_data_name, output_dir, List_Data2)
add_long_lists_to_csv(csv_file_path, column_indices2, [Step_List, Ke_Total_List , Ke_Mean_List, Temperature_Vec, Temp_Tar_List, Vx_Mean_vec , Vy_Mean_vec,Pe_T_List])

print(VmComulate)
print(len(VmComulate))

In [ ]:

def Ke_Distribution ():
  global Ke_Mean_List

  plt.close("all") #this is the line to be added
  num_steps = len(Ke_Mean_List)
  plt.plot(range(num_steps), Ke_Mean_List)
  plt.xlabel(f'''Paso x {Saving_Steps_Str}''')
  plt.ylabel('Energía cinética')
  plt.title('Distribución de la energía cinética Media')
  plt.savefig(os.path.join(output_dir, "Ke_Distribution.png"))
  plt.show()

def Vx_Mean_Distribution ():
  global Vx_Mean_vec

  plt.close("all") #this is the line to be added
  num_steps = len(Vx_Mean_vec)
  plt.plot(range(num_steps), Vx_Mean_vec)
  plt.xlabel(f'''Paso x {Saving_Steps_Str}''')
  plt.ylabel('Magnitud')
  plt.title('Distribución de Vx_prom')
  plt.savefig(os.path.join(output_dir, "Vx_Mean_Distribution.png"))
  plt.show()

def Vy_Mean_Distribution ():
  global Vy_Mean_vec

  plt.close("all") #this is the line to be added
  num_steps = len(Vy_Mean_vec)
  plt.plot(range(num_steps), Vy_Mean_vec)
  plt.xlabel(f'''Paso x {Saving_Steps_Str}''')
  plt.ylabel('Magnitud')
  plt.title('Distribución de Vy_prom')
  plt.savefig(os.path.join(output_dir, "Vy_Mean_Distribution.png"))
  plt.show()

def Temperature_Distribution ():
  global Temperature_Vec

  plt.close("all") #this is the line to be added
  num_steps = len(Temperature_Vec)
  plt.plot(range(num_steps), Temperature_Vec)
  plt.xlabel(f'''Paso x {Saving_Steps_Str}''')
  plt.ylabel('Temperatura')
  plt.title('Distribución de la Temperatura Media')
  plt.savefig(os.path.join(output_dir, "Temperature_Distribution.png"))
  plt.show()

print("Ke: ",Ke_Mean_List)
print("Ke_Mean: ",np.mean(Ke_Mean_List))
Ke_Distribution()
print("Vx_Mean_vec: ",Vx_Mean_vec)
print("Vx_Mean_vec_Mean: ",np.mean(Vx_Mean_vec))
Vx_Mean_Distribution ()
print("Vy_Mean_vec: ",Vy_Mean_vec)
print("Vy_Mean_vec_Mean: ",np.mean(Vy_Mean_vec))
Vy_Mean_Distribution ()
print("Temperature_Vec: ",Temperature_Vec)
print("Temperature_Vec_Mean: ",np.mean(Temperature_Vec))
Temperature_Distribution()


In [ ]:
import matplotlib.pyplot as pltHisto

figHisto, axHisto = pltHisto.subplots(dpi=100)  
 

pltHisto.xlabel('Speed v')
pltHisto.ylabel('Probability density')
pltHisto.hist(VmComulate, bins=60, color='y', alpha=1.0, rwidth=0.99, density=True)
pltHisto.grid(axis='y', alpha=0.75)
figHisto.savefig(os.path.join(output_dir, "Distribution_Vel_Histo1.png"))
pltHisto.show()


In [ ]:
import matplotlib.pyplot as pltHisto2

figHisto2, axHisto2 = pltHisto2.subplots(dpi=100)  

def maxwell_f( v, Temp ) : 
  return (m*v/(kB*Temp))*np.exp(-m*(v**2)/(2*kB*Temp)) 

def maxwell_fPlot( nSteps, vf, Temp ) :
  vList = np.linspace(0, vf, nSteps)
  pltHisto2.scatter(vList, maxwell_f(vList, Temp) )
  pltHisto2.show()
 

pltHisto2.xlabel('Speed v')
pltHisto2.ylabel('Probability density')
pltHisto2.hist(VmComulate, bins=60, color='y', alpha=1.0, rwidth=0.99, density=True)
pltHisto2.grid(axis='y', alpha=0.75)
maxwell_fPlot(100, max(VmComulate), Temp_Target)
figHisto2.savefig(os.path.join(output_dir, "Distribution_Vel_Histo2.png"))
pltHisto2.show()

In [ ]:
filename = "animacion2.gif"
output_path = os.path.join(output_dir, filename)

#----------------------------- Animate Graph config ----------------------------

plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150  
plt.ioff()
fig2, ax = plt.subplots(2,figsize=(4, 6),gridspec_kw={'height_ratios': [0.8, 0.2]})
frameSteps = 100

#------------ Exec movement with Animate funtion (stack movements) -------------

if ANIMATE2 == True:
    ani2 = matplotlib.animation.FuncAnimation(fig2, animate, frames=frameSteps)
    ani2.save(output_path, writer="Pillow")
    
# matplotlib.animation.FuncAnimation(fig2, animate, frames=frameSteps)


In [ ]:
def guardar_csv(arr, variable_name, output_dir):
    file_name = f"{variable_name}.csv"
    file_path = os.path.join(output_dir, file_name)
    
    with open(file_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter='|', quoting=csv.QUOTE_MINIMAL)
        
        for item in arr:
            if isinstance(item, float):
                formatted_item = "{:.9}".format(item).replace('.', ',')
                writer.writerow([formatted_item])
            else:
                writer.writerow([item])
    
guardar_csv(Ke_Mean_List, "Ke_Mean_List",output_dir)
print("Ke_Mean: ",np.mean(Ke_Mean_List))

guardar_csv(Vx_Mean_vec, "Vx_Mean_vec",output_dir)
print("Vx_Mean_vec_Mean: ",np.mean(Vx_Mean_vec))

guardar_csv(Vy_Mean_vec, "Vy_Mean_vec",output_dir)
print("Vy_Mean_vec_Mean: ",np.mean(Vy_Mean_vec))

guardar_csv(Temperature_Vec, "Temperature_Vec",output_dir)
print("Temperature_Vec_Mean: ",np.mean(Temperature_Vec))

In [ ]:
import datetime
import os
import asyncio
from telegram import Bot

async def enviar_mensaje_telegram(token, chat_id, output_dir):
    hora_finalizacion = datetime.datetime.now().strftime('%H:%M:%S')
    fecha_finalizacion = datetime.datetime.now().strftime('%Y-%m-%d')

    mensaje_completo = f'''
    {PC_ID}: La simulación ha finalizado correctamente.

    Hora de finalización: {hora_finalizacion}
    Fecha de finalización: {fecha_finalizacion}
    Directorio de resultados: {output_dir}
    '''

    bot = Bot(token=token)
    await bot.send_message(chat_id=chat_id, text=mensaje_completo)

    # Obtener la lista de archivos en el directorio
    archivos = os.listdir(output_dir)

    # Enviar cada archivo que no sea .csv por Telegram
    for archivo in archivos:
        ruta_archivo = os.path.join(output_dir, archivo)
        if not archivo.endswith('.csv'):
            await bot.send_document(chat_id=chat_id, document=open(ruta_archivo, 'rb'))

async def main():
    global output_dir
    token = "6105465299:AAG-yMSYxqHqFpoomJt1J2BNjazHukQ2cZo"
    chat_id = "5205937784"

    await enviar_mensaje_telegram(token, chat_id, output_dir)

loop = asyncio.get_event_loop()
asyncio.create_task(main())


In [ ]:
# Obtén todas las variables locales como un diccionario
variables = dict(locals())

# Nombre personalizado para el archivo CSV
nombre_archivo_csv = 'Simulation_Values.csv'

# Ruta completa del archivo CSV con el directorio y nombre personalizado
ruta_archivo_csv = f"{output_dir}/{nombre_archivo_csv}"

# Crear un DataFrame de Pandas con las variables y sus valores
df = pd.DataFrame(list(variables.items()), columns=['Variable', 'Valor'])

# Guardar el DataFrame en un archivo CSV con delimitador de coma
df.to_csv(ruta_archivo_csv, index=False)

In [ ]:
# import numpy as np

# N = 100
# kB = 1
# Temp_Target = 1.0

# xi = 1
# xi = ((xi/(2*np.pi))**2) * (2*(2*N)*kB*Temp_Target)
# xi = 1/xi

# print(xi)